In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures

churn_df = pd.read_csv('../Data/train_set_anju.csv')
churn_df = churn_df.drop('cancellation_request',axis=1)

col_names = churn_df.columns.tolist()

for i, name in enumerate(list(churn_df.columns.values)):
        if churn_df[name].dtype in [int, float]:
            churn_df[name] = churn_df.values[:, i]
        else:
            encoder = LabelEncoder()
            churn_df[name] = encoder.fit_transform(
                [str(i) for i in churn_df.values[:, i]]) 
to_drop = ['id']
churn_df = churn_df.drop(to_drop,axis=1)

churn_result = churn_df['renewed']
y = churn_result.astype(int)
churn_df = churn_df.drop('renewed',axis=1)

churn_df['account_value'] = churn_df['account_value'].replace(np.nan, 0)
churn_df = churn_df.replace(np.nan, 100000)

to_drop = ['job_count_over_client_count', 'job_count_over_invoice_count', 'job_count_over_quote_count', 
           'client_count_over_job_count','client_count_over_invoice_count', 'client_count_over_quote_count',
          'invoice_count_over_job_count', 'invoice_count_over_client_count', 'invoice_count_over_quote_count',
          'quote_count_over_job_count', 'quote_count_over_client_count', 'quote_count_over_invoice_count']

print(any(churn_df.isnull()))
churn_df = churn_df.fillna(0)
print(any(churn_df.isnull()))
churn_df[:]

True
True


,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,accepted_added_user_count,...,days_since_epayments_used,time_sheet_entries_used,users_logged_in,synced_to_quickbooks,phone_number,account_value,plan_code,annual_plan,year,month
0,42,100000.0,100000.0,100000.0,100000.0,2,20,2,2,0,...,100000.0,0,1,0,1,91,5,0,2016,4
1,323,21.0,1.0,7.0,1.0,2,37,4,1,0,...,100000.0,0,0,0,1,39,3,0,2014,10
2,621,284.0,284.0,100000.0,100000.0,2,27,4,4,3,...,100000.0,1,1,0,1,34,2,0,2014,9
3,931,129.0,136.0,122.0,129.0,1,4,1,12,11,...,100000.0,1,1,1,1,62,3,0,2016,6
4,776,215.0,305.0,26.0,286.0,2,6,1,1,1,...,100000.0,0,1,0,1,39,3,0,2016,8
5,684,6.0,27.0,6.0,62.0,2,20,2,0,0,...,100000.0,0,1,0,1,29,2,0,2015,2
6,595,152.0,98.0,150.0,100000.0,2,41,1,1,0,...,100000.0,0,0,1,1,39,3,0,2016,1
7,880,5.0,3.0,5.0,3.0,2,20,2,2,0,...,5.0,1,0,0,1,52,3,0,2016,4
8,650,5.0,11.0,17.0,160.0,2,20,2,1,1,...,100000.0,0,1,0,1,39,3,0,2015,3
9,498,6.0,5.0,6.0,5.0,2,20,2,0,0,...,10.0,0,0,0,1,44,3,0,2015,7


Ways to Preprocess Data (Brainstorming):
1. Put country/plan_code in distribution of most to least common...something with the mode 
2. 

In [2]:
 pd.isnull(churn_df).any(1).nonzero()[0]

array([], dtype=int64)

In [3]:
#Convert data to np array 
X = churn_df.as_matrix()

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

poly = PolynomialFeatures(1) 
X = poly.fit_transform(X)
print("Feature space holds %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(y))

Feature space holds 1279 observations and 57 features
Unique target labels: [0 1]


In [8]:
from sklearn.model_selection import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),shuffle=True)
    y_pred = y.copy()

    # Iterate through folds
    for i, (train_index, test_index) in enumerate(KFold(n_splits=8).split(churn_df)):
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
       
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [9]:
from sklearn.svm import SVC, LinearSVC

def accuracy(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)


print("Kernel Type, Penalty: Accuracy")
for c in range(1, 40):
    for k in ['hinge', 'squared_hinge' ]:
        print("%d %s: %.3f" % (c, k, accuracy(y, run_cv(X,y,LinearSVC, C=c, loss=k))))
    print()
    

Kernel Type, Penalty: Accuracy
1 hinge: 0.787
1 squared_hinge: 0.772

2 hinge: 0.788
2 squared_hinge: 0.771

3 hinge: 0.788
3 squared_hinge: 0.770

4 hinge: 0.788
4 squared_hinge: 0.772

5 hinge: 0.788
5 squared_hinge: 0.772

6 hinge: 0.791
6 squared_hinge: 0.772

7 hinge: 0.787
7 squared_hinge: 0.767

8 hinge: 0.791
8 squared_hinge: 0.768

9 hinge: 0.791
9 squared_hinge: 0.757

10 hinge: 0.785
10 squared_hinge: 0.757

11 hinge: 0.795
11 squared_hinge: 0.761

12 hinge: 0.790
12 squared_hinge: 0.741

13 hinge: 0.791
13 squared_hinge: 0.756

14 hinge: 0.790
14 squared_hinge: 0.753

15 hinge: 0.787
15 squared_hinge: 0.735

16 hinge: 0.783
16 squared_hinge: 0.731

17 hinge: 0.771
17 squared_hinge: 0.738

18 hinge: 0.785
18 squared_hinge: 0.742

19 hinge: 0.787
19 squared_hinge: 0.736

20 hinge: 0.783
20 squared_hinge: 0.749

21 hinge: 0.768
21 squared_hinge: 0.697

22 hinge: 0.771
22 squared_hinge: 0.724

23 hinge: 0.782
23 squared_hinge: 0.729

24 hinge: 0.780
24 squared_hinge: 0.706

25 

In [7]:
"""
None: 
0: 0.590
1: 0.748
2: 0.679
3: 0.658326817826
"""

"""
Kernel Type, Penalty: Accuracy
1 linear: 0.786
1 poly: 0.658
1 rbf: 0.748
1 sigmoid: 0.729

2 linear: 0.787
2 poly: 0.673
2 rbf: 0.751
2 sigmoid: 0.708

3 linear: 0.789
3 poly: 0.683
3 rbf: 0.749
3 sigmoid: 0.704

4 linear: 0.789
4 poly: 0.683
4 rbf: 0.747
4 sigmoid: 0.701

5 linear: 0.790
5 poly: 0.685
5 rbf: 0.740
5 sigmoid: 0.699

6 linear: 0.790
6 poly: 0.687
6 rbf: 0.740
6 sigmoid: 0.688

7 linear: 0.790
7 poly: 0.697
7 rbf: 0.742
7 sigmoid: 0.693

8 linear: 0.790
8 poly: 0.701
8 rbf: 0.737
8 sigmoid: 0.680

9 linear: 0.789
9 poly: 0.704
9 rbf: 0.738
9 sigmoid: 0.686

10 linear: 0.788
10 poly: 0.699
10 rbf: 0.735
10 sigmoid: 0.680

11 linear: 0.787
11 poly: 0.700
11 rbf: 0.732
11 sigmoid: 0.674

12 linear: 0.788
12 poly: 0.695
12 rbf: 0.734
12 sigmoid: 0.680

13 linear: 0.787
13 poly: 0.696
13 rbf: 0.732
13 sigmoid: 0.679

14 linear: 0.789
14 poly: 0.694
14 rbf: 0.730
14 sigmoid: 0.686

15 linear: 0.788
15 poly: 0.692
15 rbf: 0.728
15 sigmoid: 0.679

16 linear: 0.788
16 poly: 0.696
16 rbf: 0.726
16 sigmoid: 0.674

17 linear: 0.789
17 poly: 0.694
17 rbf: 0.726
17 sigmoid: 0.676

18 linear: 0.788
18 poly: 0.694
18 rbf: 0.726
18 sigmoid: 0.679

19 linear: 0.788
19 poly: 0.691
19 rbf: 0.725
19 sigmoid: 0.682
"""
"""
Kernel Type, Penalty: Accuracy
1 linear: 0.786
1 poly: 0.590
1 rbf: 0.701
1 sigmoid: 0.683

2 linear: 0.787
2 poly: 0.590
2 rbf: 0.726
2 sigmoid: 0.705

3 linear: 0.789
3 poly: 0.589
3 rbf: 0.737
3 sigmoid: 0.722

4 linear: 0.789
4 poly: 0.590
4 rbf: 0.746
4 sigmoid: 0.728

5 linear: 0.790
5 poly: 0.590
5 rbf: 0.747
5 sigmoid: 0.733

6 linear: 0.790
6 poly: 0.589
6 rbf: 0.753
6 sigmoid: 0.740

7 linear: 0.790
7 poly: 0.589
7 rbf: 0.756
7 sigmoid: 0.749

8 linear: 0.790
8 poly: 0.589
8 rbf: 0.755
8 sigmoid: 0.750

9 linear: 0.789
9 poly: 0.589
9 rbf: 0.758
9 sigmoid: 0.753

10 linear: 0.788
10 poly: 0.589
10 rbf: 0.761
10 sigmoid: 0.754

11 linear: 0.787
11 poly: 0.589
11 rbf: 0.761
11 sigmoid: 0.759

12 linear: 0.788
12 poly: 0.589
12 rbf: 0.762
12 sigmoid: 0.758

13 linear: 0.787
13 poly: 0.589
13 rbf: 0.764
13 sigmoid: 0.760

14 linear: 0.789
14 poly: 0.589
14 rbf: 0.763
14 sigmoid: 0.763

15 linear: 0.788
15 poly: 0.589
15 rbf: 0.765
15 sigmoid: 0.762

16 linear: 0.788
16 poly: 0.589
16 rbf: 0.765
16 sigmoid: 0.763

17 linear: 0.789
17 poly: 0.589
17 rbf: 0.765
17 sigmoid: 0.764

18 linear: 0.788
18 poly: 0.589
18 rbf: 0.765
18 sigmoid: 0.763

19 linear: 0.788
19 poly: 0.589
19 rbf: 0.768
19 sigmoid: 0.765

"""

'\nKernel Type, Penalty: Accuracy\n1 linear: 0.786\n1 poly: 0.590\n1 rbf: 0.701\n1 sigmoid: 0.683\n\n2 linear: 0.787\n2 poly: 0.590\n2 rbf: 0.726\n2 sigmoid: 0.705\n\n3 linear: 0.789\n3 poly: 0.589\n3 rbf: 0.737\n3 sigmoid: 0.722\n\n4 linear: 0.789\n4 poly: 0.590\n4 rbf: 0.746\n4 sigmoid: 0.728\n\n5 linear: 0.790\n5 poly: 0.590\n5 rbf: 0.747\n5 sigmoid: 0.733\n\n6 linear: 0.790\n6 poly: 0.589\n6 rbf: 0.753\n6 sigmoid: 0.740\n\n7 linear: 0.790\n7 poly: 0.589\n7 rbf: 0.756\n7 sigmoid: 0.749\n\n8 linear: 0.790\n8 poly: 0.589\n8 rbf: 0.755\n8 sigmoid: 0.750\n\n9 linear: 0.789\n9 poly: 0.589\n9 rbf: 0.758\n9 sigmoid: 0.753\n\n10 linear: 0.788\n10 poly: 0.589\n10 rbf: 0.761\n10 sigmoid: 0.754\n\n11 linear: 0.787\n11 poly: 0.589\n11 rbf: 0.761\n11 sigmoid: 0.759\n\n12 linear: 0.788\n12 poly: 0.589\n12 rbf: 0.762\n12 sigmoid: 0.758\n\n13 linear: 0.787\n13 poly: 0.589\n13 rbf: 0.764\n13 sigmoid: 0.760\n\n14 linear: 0.789\n14 poly: 0.589\n14 rbf: 0.763\n14 sigmoid: 0.763\n\n15 linear: 0.788\n15 